<h2><b>1.</b> Import dataset and libraries</h2>

In [ ]:
!pip install category_encoders
!pip install snapml
!pip install xgboost
!pip install lightgbm 
!pip install kagglehub

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("samiraalipour/genomics-of-drug-sensitivity-in-cancer-gdsc")

print("Path to dataset files:", path)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from category_encoders import TargetEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from snapml import SnapRandomForestRegressor, SnapBoostingMachineRegressor
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_regression
import scipy.stats as stats
from scipy.stats import skew, yeojohnson, zscore
from scipy.stats.mstats import winsorize
import time
import gc



<h2><b>2.</b> Load the Data</h2>

In [ ]:
files = os.listdir(path)
print("Files in the directory:", files)

In [ ]:
csv_path = os.path.join(path, 'GDSC_DATASET.csv')
print('Path to csv file:', csv_path)

In [ ]:
df = pd.read_csv(csv_path)
df.head()

For LN_IC50, AUC Z_SCORE: lower is better!
The target is LN_IC50



In [ ]:
df.shape

<h2><b>3.</b> Data Preparation</h2>

In [ ]:
duplicates = df.duplicated(keep='first')
sum(duplicates)

In [ ]:
df.dtypes

<h3><b>3.1</b> Let's handle missing values!</h3>

In [ ]:
corr_matrix = df.select_dtypes(include=np.number).corr()
plt.figure(figsize=(8,6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title("Correlation Matrix (numerical only)")
plt.show()

In [ ]:
df.isnull().sum()

In [ ]:
df["Cancer Type (matching TCGA label)"].value_counts()

The columns "Cancer Type (matching TCGA label)" and "TCGA_DESC" are essentially the same. I will first use them to replace each others missing values and then drop one of them.

In [ ]:
df.groupby("Cancer Type (matching TCGA label)")["TCGA_DESC"].agg([lambda x: list(x.unique()), 'nunique'])

In [ ]:
df.groupby("TCGA_DESC")["Cancer Type (matching TCGA label)"].agg([lambda x: list(x.unique()), 'nunique'])

In [ ]:
df['Cancer Type (matching TCGA label)'] = df['Cancer Type (matching TCGA label)'].fillna(df['TCGA_DESC'])

In [ ]:
df['Cancer Type (matching TCGA label)'] = df['Cancer Type (matching TCGA label)'].replace("COREAD", "COAD/READ")
df['TCGA_DESC'] = df['TCGA_DESC'].replace("COREAD", "COAD/READ")

In [ ]:
df['Cancer Type (matching TCGA label)'] = df['Cancer Type (matching TCGA label)'].replace("UNABLE TO CLASSIFY", "UNCLASSIFIED")

In [ ]:
df['TCGA_DESC'] = df['TCGA_DESC'].fillna(df['Cancer Type (matching TCGA label)'])

In [ ]:
df.isnull().sum()

As you can see above, we filled practically all the missing values from Cancer Type (matching TCGA label) column and now they are identical to TCGA_DESC. Hence, I am going to drop the latter.

In [ ]:
df['Cancer Type (matching TCGA label)'].equals( df['TCGA_DESC'])

In [ ]:
df.drop('TCGA_DESC', axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df.replace('?', np.NaN, inplace=True)

In [ ]:
df['Cancer Type (matching TCGA label)'].value_counts()

In [ ]:
df.groupby(['Cancer Type (matching TCGA label)'])['GDSC Tissue descriptor 2'].agg([lambda x: list(x.unique()), 'nunique'])

I will first replace the NaN values in 'GDSC Tissue descriptor 2' whith the most frequent value that corresponds to the 'Cancer Type (matching TCGA label)' value. Only for those values of Cancer Type that corresponf to multiple values of GDSC.

In [ ]:
filtered_df_LAML = df[df['Cancer Type (matching TCGA label)'] == 'LAML']
most_frequent_value_LAML = filtered_df_LAML['GDSC Tissue descriptor 2'].mode().iloc[0]
print("Most frequent value corresponding to LAML :", most_frequent_value_LAML)

In [ ]:
df.loc[df['Cancer Type (matching TCGA label)'] == "LAML",'GDSC Tissue descriptor 2'] = \
df.loc[df['Cancer Type (matching TCGA label)'] == "LAML",'GDSC Tissue descriptor 2'].replace(np.NaN, most_frequent_value_LAML)

In [ ]:
filtered_df_UNCLASSIFIED = df[df['Cancer Type (matching TCGA label)'] == 'UNCLASSIFIED']
most_frequent_value_UNCLASSIFIED = filtered_df_UNCLASSIFIED['GDSC Tissue descriptor 2'].mode().iloc[0]
print("Most frequent value corresponding to UNCLASSIFIED :", most_frequent_value_UNCLASSIFIED)

In [ ]:
df.loc[df['Cancer Type (matching TCGA label)'] == "UNCLASSIFIED", 'GDSC Tissue descriptor 2'] = \
df.loc[df['Cancer Type (matching TCGA label)'] == "UNCLASSIFIED", 'GDSC Tissue descriptor 2'].replace(np.NaN, most_frequent_value_UNCLASSIFIED)

I'll replace missing values from 'GDSC Tissue descriptor 2' with correspopnding value from the same column, based on the value that 'Cancer Type (matching TCGA label)' has.

In [ ]:
df['GDSC Tissue descriptor 2'] = df.groupby('Cancer Type (matching TCGA label)')['GDSC Tissue descriptor 2'] \
                                   .transform(lambda x: x.ffill().bfill())

In [ ]:
df.groupby(['Cancer Type (matching TCGA label)'])['GDSC Tissue descriptor 2'].agg([lambda x: list(x.unique()), 'nunique'])

In [ ]:
df.isnull().sum()

In [ ]:
df.groupby(['GDSC Tissue descriptor 2'])['Cancer Type (matching TCGA label)'].agg([lambda x: list(x.unique()), 'nunique'])

In [ ]:
df.groupby(['Cancer Type (matching TCGA label)'])['GDSC Tissue descriptor 1'].agg([lambda x: list(x.unique()), 'nunique'])

I will first replace the NaN values in 'GDSC Tissue descriptor 1' whith the most frequent value that corresponds to the 'Cancer Type (matching TCGA label)' value. Only for those values of Cancer Type that corresponf to multiple values of GDSC.

In [ ]:
filtered_df_UNCLASSIFIED = df[df['Cancer Type (matching TCGA label)'] == 'UNCLASSIFIED']
most_frequent_value_UNCLASSIFIED = filtered_df_UNCLASSIFIED['GDSC Tissue descriptor 1'].mode().iloc[0]
print("Most frequent value corresponding to UNCLASSIFIED :", most_frequent_value_UNCLASSIFIED)
df.loc[df['Cancer Type (matching TCGA label)'] == 'UNCLASSIFIED', 'GDSC Tissue descriptor 1'] = \
df.loc[df['Cancer Type (matching TCGA label)'] == 'UNCLASSIFIED', 'GDSC Tissue descriptor 1'] \
.replace(np.NaN, most_frequent_value_UNCLASSIFIED)

I'll replace missing values from 'GDSC Tissue descriptor 1' with correspopnding value from the same column, based on the value that 'Cancer Type (matching TCGA label)' has.

In [ ]:
df['GDSC Tissue descriptor 1'] = df.groupby('Cancer Type (matching TCGA label)')['GDSC Tissue descriptor 1'] \
                                   .transform(lambda x: x.ffill().bfill())

In [ ]:
df.isnull().sum()

Let's check the association between 'Microsatellite instability Status (MSI)' and 'Cancer Type (matching TCGA label)', as before.

In [ ]:
df.groupby("Microsatellite instability Status (MSI)")["Cancer Type (matching TCGA label)"].nunique()

We can't replace missing cancer type based on MSI because there isn't a 1-1 relationship (for each MSI we could use many possible cancer type values)

In [ ]:
df.groupby("Cancer Type (matching TCGA label)")["Microsatellite instability Status (MSI)"].agg([lambda x: list(x.unique()), 'nunique'])

However, we can replace missing values of 'MSI', according to the value of 'Cancer Type (matching TCGA label)'.

I will replace the NaN values in 'MSI' whith the most frequent value that corresponds to the 'Cancer Type (matching TCGA label)' value, for those values of Cancer Type that correspond to multiple values of 'MSI'. As for the NaN values in 'MSI' that correspond to values of Cancer Type that have single values of 'MSI', i will replace them with that single value.

In [ ]:
# Step 1: Calculate unique counts of MSI values per cancer type
msi_unique_counts = df.groupby("Cancer Type (matching TCGA label)")["Microsatellite instability Status (MSI)"].nunique()
# Step 2: Replace NaNs when there's only 1 unique non-NaN value
single_value_types = msi_unique_counts[msi_unique_counts == 1].index  # Get cancer types with 1 unique MSI value
for cancer_type in single_value_types:
    unique_value = df.loc[df['Cancer Type (matching TCGA label)'] == cancer_type, 'Microsatellite instability Status (MSI)'].dropna().iloc[0]
    df.loc[(df['Cancer Type (matching TCGA label)'] == cancer_type) & df['Microsatellite instability Status (MSI)'].isna(), 'Microsatellite instability Status (MSI)'] = unique_value
# Step 3: Replace NaNs for multi-value cancer types using the most frequent value
multi_value_types = msi_unique_counts[msi_unique_counts > 1].index  # Get cancer types with multiple MSI values
for cancer_type in multi_value_types:
    most_frequent_value = df.loc[df['Cancer Type (matching TCGA label)'] == cancer_type, 'Microsatellite instability Status (MSI)'].mode().iloc[0]
    df.loc[(df['Cancer Type (matching TCGA label)'] == cancer_type) & df['Microsatellite instability Status (MSI)'].isna(), 'Microsatellite instability Status (MSI)'] = most_frequent_value
print("NaN values successfully replaced!")


In [ ]:
df.isnull().sum()

Let's check the association between 'Microsatellite instability Status (MSI)' and 'GDSC Tissue descriptor 1', as before.

In [ ]:
df.groupby("Microsatellite instability Status (MSI)")["GDSC Tissue descriptor 1"].nunique()

Likewise, we can't replace missing 'GDSC Tissue descriptor 1' based on 'MSI' because there isn't a 1-1 relationship (for each MSI we could use many possible GDSC values)

Likewise, we can't replace missing 'GDSC Tissue descriptor 2' based on 'MSI' because there isn't a 1-1 relationship (for each MSI we could use many possible GDSC values)

In [ ]:
df.groupby("Microsatellite instability Status (MSI)")["GDSC Tissue descriptor 2"].nunique()

In [ ]:
df.groupby("GDSC Tissue descriptor 2")["GDSC Tissue descriptor 1"].agg([lambda x: list(x.unique()), 'nunique'])

In [ ]:
df[df["GDSC Tissue descriptor 1"].isna() & df["GDSC Tissue descriptor 2"].isna()]

The two GDSC columns have common missing values, so I can't use them to treplace each other.

Let's check Target and Target Pathway

In [ ]:
df.groupby("TARGET", dropna=False)['TARGET_PATHWAY'].agg([lambda x: list(x.unique()), 'nunique'])

In [ ]:
df.groupby('TARGET_PATHWAY')["TARGET"].agg([lambda x: list(x.unique()), 'nunique'])

I will replace the NaN values in 'TARGET' whith the most frequent value that corresponds to the 'TARGET_PATHWAY' value, for those values of 'TARGET_PATHWAY' that correspond to multiple values of 'TARGET'. As for the NaN values in 'TARGET' that correspond to values of 'TARGET_PATHWAY' that have single values of 'TARGET', i will replace them with that single value.



In [ ]:
# Step 1: Calculate unique counts of TARGET values per TARGET_PATHWAY
target_unique_counts = df.groupby("TARGET_PATHWAY")["TARGET"].nunique()
# Step 2: Replace NaNs when there's only 1 unique non-NaN value
single_value_types = target_unique_counts[target_unique_counts == 1].index  # Get TARGET_PATHWAY with 1 unique TARGET value
for target_path in single_value_types:
    unique_value = df.loc[df['TARGET_PATHWAY'] == target_path, 'TARGET'].dropna().iloc[0]
    df.loc[(df['TARGET_PATHWAY'] == target_path) & df['TARGET'].isna(), 'TARGET'] = unique_value
# Step 3: Replace NaNs for multi-value TARGET_PATHWAY using the most frequent value
multi_value_types = target_unique_counts[target_unique_counts > 1].index  # Get TARGET_PATHWAY with multiple TARGET values
for target_path in multi_value_types:
    most_frequent_value = df.loc[df['TARGET_PATHWAY'] == target_path, 'TARGET'].mode().iloc[0]
    df.loc[(df['TARGET_PATHWAY'] == target_path) & df['TARGET'].isna(), 'TARGET'] = most_frequent_value
print("NaN values successfully replaced!")

In [ ]:
df.isnull().sum()

The missing values in 'CNA', 'Screen Medium', 'Growth Properties', 'Gene Expression', 'Methylation' have the same value in the MSI column. Based on that value, I am going to find the most frequent value for this fatures.

In [ ]:
df[df['Growth Properties'].isna()]['Microsatellite instability Status (MSI)'].value_counts()

In [ ]:
df[df['Microsatellite instability Status (MSI)'] == 'MSS/MSI-L']['CNA'].value_counts()

In [ ]:
for feature in df[['CNA', 'Screen Medium', 'Growth Properties', 'Gene Expression', 'Methylation']]:
    df[feature] = df[feature].fillna(df[df['Microsatellite instability Status (MSI)'] == 'MSS/MSI-L'][feature].mode()[0])

In [ ]:
df.isnull().sum()

In [ ]:
df.groupby('Cancer Type (matching TCGA label)', dropna=False)["Z_SCORE"].mean()

Based on the above we see that the NaN values are closer to the NB cancer type. So I could replace the NaN values in cncer types with NB, which is a bit to average. Instead, I am going to use the the similarity of LN_IC50 for each cancer type, in a specific drug group.


In [ ]:
sns.histplot(x=df['LN_IC50'], bins=30, kde=True)
plt.show()

In [ ]:
sns.boxplot(x=df['LN_IC50'])
plt.show()

Since we have tail and outliers, I am going to use median

In [ ]:
columns_to_fill = ['Cancer Type (matching TCGA label)', 'GDSC Tissue descriptor 1', 'GDSC Tissue descriptor 2']

for col in columns_to_fill:
    median_ic50 = df.groupby(['DRUG_NAME', col])['LN_IC50'].median().reset_index()
    df = df.merge(median_ic50, on=['DRUG_NAME', col], how='left', suffixes=('', '_median'))
    df[col] = df.groupby('DRUG_NAME')[col].transform(lambda x: x.ffill().bfill())
    df.drop('LN_IC50_median', axis=1, inplace=True)

print(df[columns_to_fill].isna().sum())

In [ ]:
df.isnull().sum()

In [ ]:
df.groupby("DRUG_ID")["DRUG_NAME"].agg([lambda x: x.unique(), 'count'])

I am going to drop drug_name and cosmic_name since they're redundant and keep drug_id and cosmic_id which is more suitable for tree based models. However, I am going to transform them to category just in case.

In [ ]:
df['DRUG_ID'] = df['DRUG_ID'].astype('category')
df['COSMIC_ID'] = df['COSMIC_ID'].astype('category')

<h4>Note for later: Here I start for Linear regression, with <b>df</b> dataframe</4>

In [ ]:
df_id = df.drop(['DRUG_NAME', 'CELL_LINE_NAME'], axis=1)


In [ ]:
df_id.head()

<h3><b>3.2</b> Data Encoding</h3>

<b>3.2.1</b> I'll transform features with 2 unique values to binary features  

In [ ]:
binary_features = [col for col in df_id.columns if df_id[col].nunique() == 2]
for feature in binary_features:
    df_id[feature] = (df_id[feature] == df_id[feature].unique()[0]).astype(int)

<b>3.2.2</b> I'll transform features with 3 unique values with one-hot encoding

In [ ]:
encoder = OneHotEncoder(sparse_output=False, drop=None, handle_unknown='ignore')
encoded_features = encoder.fit_transform(df_id[['Growth Properties']])
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(['Growth Properties']))
df_id = pd.concat([df_id.drop('Growth Properties', axis=1), encoded_df], axis=1)

In [ ]:
df_id.head()

Let's check for linear relationship between the ID's and the target, to choose encoding method

In [ ]:
df_id[['DRUG_ID', 'COSMIC_ID', 'LN_IC50']].corr()

No, linear relationship. Let's check ANOVA

In [ ]:
# Filter only non-null values for COSMIC_ID
df_no_nan = df_id.dropna(subset=['COSMIC_ID', 'DRUG_ID'])

# Group `LN_IC50` by `COSMIC_ID`
categories_cosmic = df_no_nan['COSMIC_ID'].unique()
ln_ic50_groups_cosmic = [df_no_nan[df_no_nan['COSMIC_ID'] == cat]['LN_IC50'] for cat in categories_cosmic]

# Perform ANOVA for COSMIC_ID
anova_result_cosmic = stats.f_oneway(*ln_ic50_groups_cosmic)
print(f"ANOVA p-value for COSMIC_ID: {anova_result_cosmic.pvalue}")

# Group `LN_IC50` by `DRUG_ID`
categories_drug = df_no_nan['DRUG_ID'].unique()
ln_ic50_groups_drug = [df_no_nan[df_no_nan['DRUG_ID'] == cat]['LN_IC50'] for cat in categories_drug]

# Perform ANOVA for DRUG_ID
anova_result_drug = stats.f_oneway(*ln_ic50_groups_drug)
print(f"ANOVA p-value for DRUG_ID: {anova_result_drug.pvalue}")

In [ ]:
print(categories_cosmic)
print(categories_drug)

<b>3.2.3</b> Too many categories. Thus, I will use Label encoding for 'COSMIC_ID' ,	'DRUG_ID'

In [ ]:
df_encoded = df_id.copy() # = ['DRUG_ID', 'COSMIC_ID']
le_drug = LabelEncoder()
le_cosmic = LabelEncoder()
df_encoded['DRUG_ID'] = le_drug.fit_transform(df_encoded['DRUG_ID'])
df_encoded['COSMIC_ID'] = le_cosmic.fit_transform(df_encoded['COSMIC_ID'])

<b>3.2.4</b> I'll use target encoding for the rest (high-cardinal) features, since we have a big data set and I am going to use tree based regression, along with KFold to reduce overfitting.

In [ ]:
categorical_features = ['GDSC Tissue descriptor 1',
                        'GDSC Tissue descriptor 2', 'Cancer Type (matching TCGA label)',
                        'TARGET', 'TARGET_PATHWAY']

# Define the target variable
target = 'LN_IC50'
# Step 1: Split the dataset into train & test **before encoding**
train_df, test_df = train_test_split(df_encoded, test_size=0.2, random_state=42)

# Number of folds
K = 5
kf = KFold(n_splits=K, shuffle=True, random_state=42)

# Make copies to avoid modifying the original dataset
train_encoded = train_df.copy()
test_encoded = test_df.copy()

# Initialize dictionary to store target means
means_dict = {}

# Apply K-Fold Target Encoding on the training set
for col in categorical_features:
    train_encoded[col + '_encoded'] = np.nan  # Create new column for encoding

    for train_idx, val_idx in kf.split(train_encoded):
        train_fold = train_encoded.iloc[train_idx]
        val_fold = train_encoded.iloc[val_idx]

        # Compute mean target for each category (excluding current fold)
        fold_means = train_fold.groupby(col)[target].mean()

        # Apply encoding to validation fold
        train_encoded.iloc[val_idx, train_encoded.columns.get_loc(col + '_encoded')] = val_fold[col].map(fold_means)

    # Store the global mean for test-time replacement
    means_dict[col] = train_encoded.groupby(col)[target].mean()

# Step 3: Apply Target Encoding to the test set using learned means from training
for col in categorical_features:
    test_encoded[col + '_encoded'] = test_encoded[col].map(means_dict[col])  # Apply learned means
    test_encoded[col + '_encoded'] = test_encoded[col + '_encoded'].fillna(train_encoded[col + '_encoded'].mean())  # Fill unseen categories with global mean

# Drop original categorical columns
train_encoded = train_encoded.drop(columns=categorical_features)
test_encoded = test_encoded.drop(columns=categorical_features)

# Check results
print(train_encoded.head())
print(test_encoded.head())

<h2><b>4.</b> Data Visualisation</h2>

<h3><b>4.1</b> Numerical</h3>

In [ ]:
for feat in ['LN_IC50' , 'AUC', 'Z_SCORE']:
  plt.figure(figsize=(8,6))
  sns.histplot(x=train_encoded[feat], bins=30, kde=True)
  plt.xlabel(feat)
  plt.ylabel('Frequency')
  plt.title(f'Distribution of {feat}')
  plt.show()



In [ ]:
for feat in ['AUC', 'Z_SCORE', 'TARGET_encoded', 'TARGET_PATHWAY_encoded']:
  plt.figure(figsize=(8,6))
  sns.regplot(x=feat, y='LN_IC50', line_kws={'color':'red'}, data=train_encoded)
  plt.xlabel(feat)
  plt.ylabel('LN_IC50')
  plt.title(f'LN_IC50 vs {feat}')
  plt.show()
  pearson_coef, p_value = stats.pearsonr(train_encoded[feat], train_encoded['LN_IC50'])
  print(f"The Pearson Correlation Coefficient of {feat} with LN_IC50 is: {pearson_coef}, with a P-value of: P = {p_value}")

<h3><b>4.2</b> Categorical</h3>

In [ ]:
for feat in ['Microsatellite instability Status (MSI)', 'Screen Medium',  'CNA', 'Gene Expression', \
             'Methylation', 'Growth Properties_Adherent', 'Growth Properties_Semi-Adherent', \
             'Growth Properties_Suspension']:
  plt.figure(figsize=(8,6))
  sns.boxplot(x=feat, y='LN_IC50', data=train_encoded)
  plt.xlabel(feat)
  plt.ylabel('LN_IC50')
  plt.title(f'Boxplot of {feat} vs LN_IC50')
  plt.show()

As we can see many outliers and great overlap, so linear regression isn't the best choice

In [ ]:
for feat in ['GDSC Tissue descriptor 1',  'GDSC Tissue descriptor 2', \
             'Cancer Type (matching TCGA label)', 'TARGET', 'TARGET_PATHWAY']:
  plt.figure(figsize=(16,9))
  sns.barplot(x='LN_IC50', y=feat , orient='h', data=df)
  plt.xlabel('LN_IC50')
  plt.ylabel(feat)
  plt.title(f'Barplot of {feat} vs LN_IC50')
  plt.show()

In [ ]:
cancer_types = df.groupby('Cancer Type (matching TCGA label)')['LN_IC50'].mean().sort_values(ascending=False)
plt.figure(figsize=(16,9))
sns.barplot(x=cancer_types.index, y=cancer_types.values)
plt.xlabel('Cancer Type')
plt.ylabel('Mean LN_IC50')
plt.title('Mean LN_IC50 by Cancer Type')
plt.xticks(rotation=90)
plt.show()

In [ ]:
tissue_types = df.groupby('GDSC Tissue descriptor 1')['AUC'].median().sort_values(ascending=False)
plt.figure(figsize=(16,9))
sns.boxplot(x=tissue_types.index, y=tissue_types.values)
plt.xlabel('GDSC Tissue descriptor 1')
plt.ylabel('Median AUC')
plt.title('Median AUC by GDSC Tissue descriptor 1')
plt.xticks(rotation=90)
plt.show()


In [ ]:
top_targets = df['TARGET'].value_counts().nlargest(10)
plt.figure(figsize=(12,7))
sns.barplot(x=top_targets.values, y=top_targets.index,\
            orient='h', hue=top_targets.index)
plt.xlabel('Count')
plt.ylabel('Target')
plt.title('Top 10 Targets by Count')
plt.show()


In [ ]:
pathway_counts = df['TARGET_PATHWAY'].value_counts()
plt.figure(figsize=(12,7))
sns.barplot(y=pathway_counts.index, x=pathway_counts.values, orient='h', hue=pathway_counts.index)
plt.xlabel('Count')
plt.ylabel('Target Pathway')
plt.title('Count of Target Pathways')
plt.show()

In [ ]:
pivot_table = pd.pivot_table(df, values='LN_IC50', index='Cancer Type (matching TCGA label)', columns='DRUG_NAME', aggfunc='mean')
plt.figure(figsize=(20,16))
sns.heatmap(pivot_table, cmap='coolwarm', annot=False)
plt.xlabel('DRUG_NAME')
plt.ylabel('Cancer Type (matching TCGA label)')
plt.title('Heatmap of LN_IC50 across Cancer types and Drug Name')
plt.xticks(rotation=90)
plt.show()

Drugs effectiveness comparison for each tissue and specific condition

In [ ]:
df_pivot = df.pivot_table(index=['GDSC Tissue descriptor 1', 'GDSC Tissue descriptor 2', 'Cancer Type (matching TCGA label)'], columns='DRUG_NAME', values='LN_IC50', aggfunc='median')
plt.figure(figsize=(20,12))
sns.heatmap(df_pivot, cmap='coolwarm', annot=False)
plt.title('Median LN_IC50 by GDSC Tissue descriptor 1, GDSC Tissue descriptor 2, and Cancer Type')
plt.show()

But that is too crowded. Lets's selcet the 10 most effective in each cancer type

In [ ]:
top_n = 10
top_drugs_per_cancer = (
df.groupby(['GDSC Tissue descriptor 1', 'GDSC Tissue descriptor 2',\
                                    'Cancer Type (matching TCGA label)',\
                                    'DRUG_NAME'])['LN_IC50'].median().reset_index().sort_values(['GDSC Tissue descriptor 1', 'GDSC Tissue descriptor 2',\
                                    'Cancer Type (matching TCGA label)', 'Cancer Type (matching TCGA label)',\
                                    'LN_IC50']).groupby(['GDSC Tissue descriptor 1', 'GDSC Tissue descriptor 2',\
                                    'Cancer Type (matching TCGA label)','Cancer Type (matching TCGA label)']).head(top_n)
)
#print(top_drugs_per_cancer)
df_pivot = top_drugs_per_cancer.pivot_table(index=['GDSC Tissue descriptor 1', 'GDSC Tissue descriptor 2', 'Cancer Type (matching TCGA label)'], columns='DRUG_NAME', values='LN_IC50', aggfunc='median')
#print(df_pivot)
plt.figure(figsize=(20,12))
sns.heatmap(df_pivot, cmap='coolwarm', annot=False)
plt.title('Top 10 Most Effective Drugs per Cancer Type')
plt.xlabel('Drug Name')
plt.ylabel('Cancer Type / Tissue Descriptor')
plt.xticks(rotation=90)
plt.show()

<h2><b>5.</b> Data Analysis</h2>

<h3><b>5.1</b> Correlation </h3>

In [ ]:
corr_matrix = train_encoded.corr()
plt.figure(figsize=(14,12))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()

<h3><b>5.2</b> Find Outliers</h3>

<h4><b>5.2.1</b> Based on all drugs</h4> 

In [ ]:
def find_outliers_by_drug(df, numeric_cols=['LN_IC50', 'AUC', 'Z_SCORE']):
    outliers = {}

    for drug in df['DRUG_ID'].unique():
        drug_data = df[df['DRUG_ID'] == drug]
        drug_outliers = {}

        for col in numeric_cols:
            v = drug_data[col]
            q1 = v.quantile(0.25)
            q3 = v.quantile(0.75)
            iqr = q3 - q1
            lower_bound = q1 - 1.5 * iqr
            upper_bound = q3 + 1.5 * iqr
            outliers_count = ((v < lower_bound) | (v > upper_bound)).sum()
            perc = outliers_count * 100.0 / len(drug_data)
            drug_outliers[col] = (perc, outliers_count)
            print(f"Drug: {drug}, Column: {col} outliers = {perc:.2f}% ({outliers_count} out of {len(drug_data)})")

        outliers[drug] = drug_outliers

    return outliers

# Find outliers in the DataFrame for each drug
outliers = find_outliers_by_drug(train_encoded)

<h4><b>5.2.2</b> Based on Cancer type</h4> 

In [ ]:
def find_outliers_by_cancer(df, numeric_cols=['LN_IC50', 'AUC', 'Z_SCORE']):
    outliers = {}

    for cancer in df['Cancer Type (matching TCGA label)'].unique():
        cancer_data = df[df['Cancer Type (matching TCGA label)'] == cancer]
        cancer_outliers = {}

        for col in numeric_cols:
            v = cancer_data[col]
            q1 = v.quantile(0.25)
            q3 = v.quantile(0.75)
            iqr = q3 - q1
            lower_bound = q1 - 1.5 * iqr
            upper_bound = q3 + 1.5 * iqr
            outliers_count = ((v < lower_bound) | (v > upper_bound)).sum()
            perc = outliers_count * 100.0 / len(cancer_data)
            cancer_outliers[col] = (perc, outliers_count)
            print(f"Cancer: {cancer}, Column: {col} outliers = {perc:.2f}% ({outliers_count} out of {len(cancer_data)})")

        outliers[cancer] = cancer_outliers

    return outliers

# Find outliers in the DataFrame for each cancer
outliers = find_outliers_by_cancer(df)

<h4><b>5.2.3</b> Based on drugs used in each medical condition</h4> 

In [ ]:
def find_outliers_by_medical_condition(df, group_cols, numeric_cols):

    outliers = {}

    # Group by medical condition (three categorical columns)
    grouped = df.groupby(group_cols)

    for group, subset in grouped:
        condition_key = "_".join(map(str, group))  # Unique key for each group
        outliers[condition_key] = {}
        unique_drugs = subset['DRUG_NAME'].unique()  # Get unique drugs in this condition

        for drug in unique_drugs:
            drug_data = subset[subset['DRUG_NAME'] == drug]
            drug_outliers = {}

            for col in numeric_cols:
                v = drug_data[col]
                q1 = v.quantile(0.25)
                q3 = v.quantile(0.75)
                iqr = q3 - q1
                lower_bound = q1 - 1.5 * iqr
                upper_bound = q3 + 1.5 * iqr

                outliers_count = ((v < lower_bound) | (v > upper_bound)).sum()
                perc = outliers_count * 100.0 / len(drug_data)

                drug_outliers[col] = (perc, outliers_count)
                print(f"Medical Condition: {condition_key}, Drug: {drug}, Column: {col} → Outliers: {perc:.2f}% ({outliers_count})")

            outliers[condition_key][drug] = drug_outliers

    return outliers

# Columns to group by (medical condition) and numerical columns for outlier detection
group_columns = ['GDSC Tissue descriptor 1', 'GDSC Tissue descriptor 2', 'Cancer Type (matching TCGA label)']
numeric_columns = ['LN_IC50', 'AUC', 'Z_SCORE']

# Run the function
outliers_by_condition = find_outliers_by_medical_condition(df, group_columns, numeric_columns)

From the three previous methods of calculating outliers, I prefer the last one because it is more specific for the drugs per condition and thus it preserves more information

<h3><b>5.3</b> Calculate skewness</h3>

<h4><b>5.3.1</b> Based on all drugs</h4> 

def check_outliers_and_skewness(df, numeric_cols):
    skewness_info = {}
    outlier_info = {}

    for drug in df['DRUG_NAME'].unique()[:2]:  # Only first two drugs
        drug_data = df[df['DRUG_NAME'] == drug]
        skewness_info[drug] = {}
        outlier_info[drug] = {}

        for col in numeric_cols:
            # Calculate skewness
            col_skewness = skew(drug_data[col].dropna())
            skewness_info[drug][col] = col_skewness

            # Identify outliers using IQR method
            Q1 = drug_data[col].quantile(0.25)
            Q3 = drug_data[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            outliers = drug_data[(drug_data[col] < lower_bound) | (drug_data[col] > upper_bound)][col]
            outlier_info[drug][col] = outliers

            # Plot distribution and box plot
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
            fig.suptitle(f'Distribution and Box Plot of {col} for {drug}')

            # Histogram
            sns.histplot(drug_data[col], kde=True, ax=ax1)
            ax1.axvline(lower_bound, color='r', linestyle='--', label='IQR bounds')
            ax1.axvline(upper_bound, color='r', linestyle='--')
            ax1.set_title(f'Histogram (Skewness: {col_skewness:.2f})')
            ax1.legend()

            # Box plot
            sns.boxplot(x=drug_data[col], ax=ax2)
            ax2.set_title('Box Plot')

            plt.tight_layout()
            plt.show()

            print(f"Drug: {drug}, Column: {col}")
            print(f"Skewness: {col_skewness:.2f}")
            print(f"Number of outliers: {len(outliers)}")
            print("-" * 50)

    return skewness_info, outlier_info


numeric_cols = ['LN_IC50', 'AUC', 'Z_SCORE']
skewness_info, outlier_info = check_outliers_and_skewness(df, numeric_cols)

<h4><b>5.3.2</b> Based on Cancer type</h4> 

In [ ]:
def check_outliers_and_skewness_cancer(df, numeric_cols):
    skewness_info = {}
    outlier_info = {}

    for cancer in df['Cancer Type (matching TCGA label)'].unique()[:2]:  # Only first two drugs
        cancer_data = df[df['Cancer Type (matching TCGA label)'] == cancer]
        skewness_info[cancer] = {}
        outlier_info[cancer] = {}

        for col in numeric_cols:
            # Calculate skewness
            col_skewness = skew(cancer_data[col].dropna())
            skewness_info[cancer][col] = col_skewness

            # Identify outliers using IQR method
            Q1 = cancer_data[col].quantile(0.25)
            Q3 = cancer_data[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            outliers = cancer_data[(cancer_data[col] < lower_bound) | (cancer_data[col] > upper_bound)][col]
            outlier_info[cancer][col] = outliers

            # Plot distribution and box plot
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
            fig.suptitle(f'Distribution and Box Plot of {col} for {cancer}')

            # Histogram
            sns.histplot(cancer_data[col], kde=True, ax=ax1)
            ax1.axvline(lower_bound, color='r', linestyle='--', label='IQR bounds')
            ax1.axvline(upper_bound, color='r', linestyle='--')
            ax1.set_title(f'Histogram (Skewness: {col_skewness:.2f})')
            ax1.legend()

            # Box plot
            sns.boxplot(x=cancer_data[col], ax=ax2)
            ax2.set_title('Box Plot')

            plt.tight_layout()
            plt.show()

            print(f"Cancer: {cancer}, Column: {col}")
            print(f"Skewness: {col_skewness:.2f}")
            print(f"Number of outliers: {len(outliers)}")
            print("-" * 50)

    return skewness_info, outlier_info


numeric_cols = ['LN_IC50', 'AUC', 'Z_SCORE']
skewness_info, outlier_info = check_outliers_and_skewness_cancer(df, numeric_cols)

<h4><b>5.3.3</b> Based on drugs used in each medical condition</h4> 

In [ ]:
def check_outliers_and_skewness_by_condition(df, group_cols, numeric_cols):

    skewness_info = {}

    outlier_info = {}


    grouped = df.groupby(group_cols)


    for group, subset in grouped:

        condition_key = "_".join(map(str, group))  # Unique key for each condition

        skewness_info[condition_key] = {}

        outlier_info[condition_key] = {}


        # Select first two drugs in this condition

        unique_drugs = subset['DRUG_NAME'].unique()[:2]


        for drug in unique_drugs:

            drug_data = subset[subset['DRUG_NAME'] == drug]

            skewness_info[condition_key][drug] = {}

            outlier_info[condition_key][drug] = {}


            for col in numeric_cols:

                # Calculate skewness

                col_skewness = skew(drug_data[col].dropna())

                skewness_info[condition_key][drug][col] = col_skewness


                # Identify outliers using IQR method

                Q1 = drug_data[col].quantile(0.25)

                Q3 = drug_data[col].quantile(0.75)

                IQR = Q3 - Q1

                lower_bound = Q1 - 1.5 * IQR

                upper_bound = Q3 + 1.5 * IQR

                outliers = drug_data[(drug_data[col] < lower_bound) | (drug_data[col] > upper_bound)][col]

                outlier_info[condition_key][drug][col] = outliers


                # Plot distribution and box plot

                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

                fig.suptitle(f'Distribution and Box Plot of {col} for {drug} (Condition: {condition_key})')


                # Histogram

                sns.histplot(drug_data[col], kde=True, ax=ax1)

                ax1.axvline(lower_bound, color='r', linestyle='--', label='IQR bounds')

                ax1.axvline(upper_bound, color='r', linestyle='--')

                ax1.set_title(f'Histogram (Skewness: {col_skewness:.2f})')

                ax1.legend()


                # Box plot

                sns.boxplot(x=drug_data[col], ax=ax2)

                ax2.set_title('Box Plot')


                plt.tight_layout()

                plt.show()


                print(f"Condition: {condition_key}, Drug: {drug}, Column: {col}")

                print(f"Skewness: {col_skewness:.2f}")

                print(f"Number of outliers: {len(outliers)}")

                print("-" * 50)


    return skewness_info, outlier_info



# Columns to group by (medical condition) and numeric columns for analysis

group_columns = ['GDSC Tissue descriptor 1', 'GDSC Tissue descriptor 2', 'Cancer Type (matching TCGA label)']

numeric_columns = ['LN_IC50', 'AUC', 'Z_SCORE']


# Run the function

skewness_info, outlier_info = check_outliers_and_skewness_by_condition(df, group_columns, numeric_columns)

As we can see again the method that checks the outliers and skewness of dtugs grouped by condition is better (less outliers/more info/better explicability)

<h3><b>5.4</b> Handle outliers and skewnness</h3>

In [ ]:
def handle_outliers_and_skewness(df, numeric_cols, yeojohnson_lambdas=None, fit_lambda=True):

    """

    Handles outliers and skewness for numeric columns while grouping by encoded tissue/cancer type columns.

    - Uses Winsorization to cap extreme values.

    - Applies Z-score normalization.

    - Fits Yeo-Johnson transformation on train and applies same λ to test (avoids data leakage).


    Parameters:

        df (pd.DataFrame): The dataset to process.

        numeric_cols (list): List of numeric columns to process.

        yeojohnson_lambdas (dict, optional): Dictionary storing λ values for test application.

        fit_lambda (bool): Whether to compute λ (True for train, False for test).


    Returns:

        df_processed (pd.DataFrame): The processed dataset.

        yeojohnson_lambdas (dict, optional): The λ values for reuse (if training).

    """

    df = df.copy()  # Avoid modifying original data
    yeojohnson_lambdas = {} if fit_lambda else yeojohnson_lambdas  # Store λ values only when fitting

    # Group by descriptors and process per drug
    grouped = df.groupby(["GDSC Tissue descriptor 1_encoded", "GDSC Tissue descriptor 2_encoded", "Cancer Type (matching TCGA label)_encoded"])

    for (desc1, desc2, cancer_type), group in grouped:
        for drug in group["DRUG_ID"].unique():
            drug_data = group[group["DRUG_ID"] == drug].copy()

            for col in numeric_cols:
                values = drug_data[col].dropna()

                if values.nunique() < 5:  # Skip if too few unique values
                    continue

                # Winsorization: Define specific limits per column
                winsorization_limits = {
                    "LN_IC50": (0.01, 0.01),
                    "AUC": (0.05, 0.05),
                    "Z_SCORE": (0.01, 0.01)
                }
                # Apply column-specific Winsorization:
                lower, upper = winsorization_limits.get(col, (0.05, 0.05))
                capped = winsorize(values, (lower, upper))

                drug_data.loc[values.index, col] = pd.Series(capped, index=values.index)

                # Apply Z-score normalization
                drug_data[col] = zscore(drug_data[col])

                # Recalculate skewness after winsorization
                new_skewness = skew(drug_data[col].dropna())

                # Apply Yeo-Johnson only if data is highly skewed
                if abs(new_skewness) > 1:
                    if fit_lambda:
                        # Fit Yeo-Johnson on train and store λ
                        transformed, lambda_ = yeojohnson(drug_data[col].dropna())
                        yeojohnson_lambdas[(desc1, desc2, cancer_type, drug, col)] = lambda_
                    else:
                        # Apply the precomputed λ from training
                        lambda_ = yeojohnson_lambdas.get((desc1, desc2, cancer_type, drug, col), None)
                        if lambda_ is None:
                            continue  # Skip if λ is missing (unseen data)

                        transformed = yeojohnson(drug_data[col].dropna(), lmbda=lambda_)
                    drug_data.loc[drug_data[col].dropna().index, col] = transformed

            # Update the original dataframe
            df.loc[drug_data.index, numeric_cols] = drug_data[numeric_cols]

    return (df, yeojohnson_lambdas) if fit_lambda else df


#  Step 1: Process Train and Store λ
numeric_cols = ["AUC", "Z_SCORE"] #"LN_IC50",
train_processed, stored_lambdas = handle_outliers_and_skewness(train_encoded, numeric_cols, fit_lambda=True)

#  Step 2: Process Test Using Stored λ from Train
test_processed = handle_outliers_and_skewness(test_encoded, numeric_cols, yeojohnson_lambdas=stored_lambdas, fit_lambda=False)

In [ ]:
print(f"Initial Skewness: {skew(df_id['LN_IC50']):.4f}")
print(f"Final Skewness: {skew(processed_data['LN_IC50']):.4f}")
print(f"Skewness difference: {(skew(df_id['LN_IC50']) - skew(processed_data['LN_IC50'])):.4f}")

There isn't much imporvement in outliers and skewness after applying the various handling methods, beacuse the grouping of drugs per condition already minimizes them. Hence, I go forth without using the previous functions.

<h2><b>6.</b> Machine Learning</h2>

<h3><b>6.1</b> Random Forest</h3>

We can always use more parameters in grid search which will slightly improve the result, but for memory reasons I only stick to one set of parameters. The same goes for cross validation folds.

In [ ]:

X_train = train_encoded.drop('LN_IC50', axis=1)
y_train = train_encoded['LN_IC50']
X_test = test_encoded.drop('LN_IC50', axis=1)
y_test = test_encoded['LN_IC50']
Input = [('polynomial', PolynomialFeatures(degree=2, include_bias=False)),
         ('scaler', StandardScaler()), ('model', RandomForestRegressor(random_state=42))]
pipe = Pipeline(Input)
param_grid = {
    'model__n_estimators': [100],
    'model__max_depth': [None],
    'model__min_samples_split': [2],
    'model__min_samples_leaf': [1],
    'model__max_features': ['sqrt']
}
gridsearch = GridSearchCV(pipe, param_grid, cv=3, scoring='r2', n_jobs=-1, verbose=1)
start_time = time.time()
gridsearch.fit(X_train, y_train)
end_time = time.time()
time_before = end_time - start_time
print(f"Training time before feature selection: {time_before:.4f} seconds")
print("Best parameters:", gridsearch.best_params_)
print("Best score:", gridsearch.best_score_)
best_model = gridsearch.best_estimator_
kf = KFold(n_splits=3, shuffle=True, random_state=42)
cv_scores = cross_val_score(best_model, X_train, y_train, cv=kf, scoring='r2', n_jobs=-1)
print(f'R^2 scores for each fold: {np.round(cv_scores, 4)}')
print(f'Mean R^2 Score: {np.mean(np.round(cv_scores, 4))}')
print(f'Standard Deviation of R^2 σ: {np.std(np.round(cv_scores, 4))}')
r2_test = best_model.score(X_test, y_test)
test_preds = best_model.predict(X_test)
rmse_test = np.sqrt(mean_squared_error(y_test, test_preds))
print(f"Final Test R^2 score: {r2_test:.4f}")
print(f"Final Test RMSE: {rmse_test:.4f}")


In [ ]:
#The same as before but without cross_val_score
#As it turns out (obviously) cross_val_score is redundunt,
# since GridSearchCV already performs cross validation

X_train = train_encoded.drop('LN_IC50', axis=1)
y_train = train_encoded['LN_IC50']
X_test = test_encoded.drop('LN_IC50', axis=1)
y_test = test_encoded['LN_IC50']

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

Input = [('polynomial', PolynomialFeatures(degree=2, include_bias=False)),
         ('scaler', StandardScaler()), ('model', RandomForestRegressor(random_state=42))]
pipe = Pipeline(Input)
param_grid = {
    'model__n_estimators': [600],
    'model__max_depth': [None],
    'model__min_samples_split': [2],
    'model__min_samples_leaf': [1],
    'model__max_features': ['sqrt']
}
gridsearch = GridSearchCV(pipe, param_grid, cv=3, scoring='r2', n_jobs=-1, verbose=1)
start_time = time.time()
gridsearch.fit(X_train, y_train)
end_time = time.time()
time_before = end_time - start_time
print(f"Training time before feature selection: {time_before:.4f} seconds")
print("Best parameters:", gridsearch.best_params_)
print("Best score:", gridsearch.best_score_)
best_model = gridsearch.best_estimator_
r2_test = best_model.score(X_test, y_test)
test_preds = best_model.predict(X_test)
rmse_test = np.sqrt(mean_squared_error(y_test, test_preds))
print(f"Final Test R^2 score: {r2_test:.4f}")
print(f"Final Test RMSE: {rmse_test:.4f}")


<h4><b>6.1.1</b> Feature Importance</h4>

In [ ]:
feature_importance = best_model.named_steps['model'].feature_importances_
feature_names = best_model.named_steps['polynomial'].get_feature_names_out(X_train.columns)
print(f"Length of feature importance: {len(feature_importance)}")
print(f"Length of feature names: {len(feature_names)}")
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
plt.figure(figsize=(25, 15))
sns.barplot(x='Importance', y='Feature', hue='Feature', data=feature_importance_df, palette='viridis')
plt.title('Random Forest Feature Importance', fontsize=16)
plt.xlabel('Importance', fontsize=14)
plt.ylabel('Feature', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

<h4><b>6.1.2</b> Remove Low-Importance features and retrain</h4>

In [ ]:
del df, df_id, train_encoded, test_encoded, filtered_df_LAML, filtered_df_UNCLASSIFIED, target_unique_counts
gc.collect()

In [ ]:
#  Apply PolynomialFeatures (Keep degree=2 for fair comparison)
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# Get Correct Feature Names
feature_names_poly = poly.get_feature_names_out(X_train.columns)

# Convert to DataFrame
X_train_poly_df = pd.DataFrame(X_train_poly, columns=feature_names_poly, index=X_train.index)
X_test_poly_df = pd.DataFrame(X_test_poly, columns=feature_names_poly, index=X_test.index)

#  Measure Training Time BEFORE Feature Selection.
# Not enough memory to perform the fit again! I'll use the time_before from the previous run
#start_time = time.time()
#best_model.fit(X_train_poly_df, y_train)  # Train with ALL features
#end_time = time.time()
#time_before = end_time - start_time
#print(f" Training Time BEFORE Feature Selection: {time_before:.4f} seconds")

#  Remove Low-Importance Features (AFTER PolynomialFeatures)
importance_threshold = 0.001  # Adjust as needed
low_importance_features = feature_importance_df[feature_importance_df["Importance"] < importance_threshold]["Feature"].tolist()

X_train_filtered = X_train_poly_df.drop(columns=low_importance_features)
X_test_filtered = X_test_poly_df.drop(columns=low_importance_features)

# Measure Training Time AFTER Feature Selection
start_time = time.time()
best_model.fit(X_train_filtered, y_train)  # Train with REDUCED features
end_time = time.time()
time_after = end_time - start_time
print(f"Training Time AFTER Feature Selection: {time_after:.4f} seconds")

#  Compare Time Reduction
time_reduction = ((time_before - time_after) / time_before) * 100
print(f"Time Reduction: {time_reduction:.2f}%")

#  Evaluate on Test Set (AFTER Feature Selection)
r2_test_filtered = best_model.score(X_test_filtered, y_test)
test_preds_filtered = best_model.predict(X_test_filtered)
rmse_test_filtered = np.sqrt(mean_squared_error(y_test, test_preds_filtered))

print(f"Final Test R² After Feature Selection: {r2_test_filtered:.4f}")
print(f"Final Test RMSE After Feature Selection: {rmse_test_filtered:.4f}")

:There is a slight accuracy improvement, but much worse time.

In [ ]:
del X_train_poly, X_test_poly, X_train_poly_df, X_test_poly_df, X_train_filtered, X_test_filtered
gc.collect()

Let's try SnapML to speed up the calculation

<h3><b>6.2</b> SnapML</h3>

In [ ]:
#SnapML
X_train = train_encoded.drop('LN_IC50', axis=1)
y_train = train_encoded['LN_IC50']
X_test = test_encoded.drop('LN_IC50', axis=1)
y_test = test_encoded['LN_IC50']

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

Input = [
    ('polynomial', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler()),
    ('model', SnapRandomForestRegressor(random_state=42))
]
pipe = Pipeline(Input)
param_grid = {
    'model__n_estimators': [100],
    'model__max_depth': [None],
    'model__min_samples_leaf': [1],
    'model__max_features': ['sqrt']
}
gridsearch = GridSearchCV(pipe, param_grid, cv=3, scoring='r2', n_jobs=-1, verbose=1)
start_time = time.time()
gridsearch.fit(X_train, y_train)
end_time = time.time()
time_before = end_time - start_time
print(f"Training time before feature selection with SnapML: {time_before:.4f} seconds")
print("Best parameters SnapML:", gridsearch.best_params_)
print("Best score SnapML:", gridsearch.best_score_)
best_model = gridsearch.best_estimator_
r2_test = best_model.score(X_test, y_test)
test_preds = best_model.predict(X_test)
rmse_test = np.sqrt(mean_squared_error(y_test, test_preds))
print(f"Final Test R^2 score with SnapML: {r2_test:.4f}")
print(f"Final Test RMSE with SnapML: {rmse_test:.4f}")

<h3><b>6.3</b> Decision Tree</h3>

In [ ]:
X_train = train_encoded.drop('LN_IC50', axis=1)
y_train = train_encoded['LN_IC50']
X_test = test_encoded.drop('LN_IC50', axis=1)
y_test = test_encoded['LN_IC50']

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

Input = [
    ('polynomial', PolynomialFeatures(degree=2, include_bias=False)),
    ('scaler', StandardScaler()),
    ('model', DecisionTreeRegressor(random_state=42))
]
pipe = Pipeline(Input)
param_grid = {
    'model__max_depth': [None],
    'model__min_samples_split': [2],
    'model__min_samples_leaf': [1],
    'model__max_features': ['sqrt']
}
gridsearch = GridSearchCV(pipe, param_grid, cv=3, scoring='r2', n_jobs=-1, verbose=1)

start_time = time.time()
gridsearch.fit(X_train, y_train)
end_time = time.time()
time_before = end_time - start_time
print(f"Training time before feature selection with Decision Tree: {time_before:.4f} seconds")
best_model = gridsearch.best_estimator_
print("Best parameters:", gridsearch.best_params_)
print("Best train R^2 score:", gridsearch.best_score_)

test_preds = best_model.predict(X_test)
r2_test = r2_score(y_test, test_preds)
rmse_test = np.sqrt(mean_squared_error(y_test, test_preds))

print(f"Final Test R^2 score: {r2_test:.4f}")
print(f"Final Test RMSE: {rmse_test:.4f}")

The faster, but not as good as SnapML. Also, using feature importance improved the R^2 score slightly from 0.9757 to 0.9875.

<h3><b>6.4</b> Linear Regression</h3>

<h4><b>6.4.1</b> Polynomial Regression</h4>

In [ ]:
X_train = train_encoded.drop('LN_IC50', axis=1)
y_train = train_encoded['LN_IC50']
X_test = test_encoded.drop('LN_IC50', axis=1)
y_test = test_encoded['LN_IC50']

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

Input = [
        ('polynomial', PolynomialFeatures(include_bias=False)),
        ('scaler', StandardScaler()),
        ('model', LinearRegression())
        ]

pipe = Pipeline(Input)
param_grid = {'polynomial__degree':[2, 3]}
gridsearch = GridSearchCV(pipe, param_grid, cv=3, scoring='r2', n_jobs=-1, verbose=1)

start_time = time.time()
gridsearch.fit(X_train, y_train)
end_time = time.time()
time_before = end_time - start_time
print(f"Training time before feature selection with Multi-Linear Polynomial Regrssion: {time_before:.4f} seconds")
best_model = gridsearch.best_estimator_
print("Best parameters:", gridsearch.best_params_)
print("Best train R^2 score:", gridsearch.best_score_)

test_preds = best_model.predict(X_test)
r2_test = r2_score(y_test, test_preds)
rmse_test = np.sqrt(mean_squared_error(y_test, test_preds))

print(f"Final Test R^2 score: {r2_test:.4f}")
print(f"Final Test RMSE: {rmse_test:.4f}")

As expected for a non-linear dataframe, Linear Regression is much worse than Trees, even with Polynomial Features.

<h4><b>6.4.2</b> Ridge Regression</h4>

In [ ]:
X_train = train_encoded.drop('LN_IC50', axis=1)
y_train = train_encoded['LN_IC50']
X_test = test_encoded.drop('LN_IC50', axis=1)
y_test = test_encoded['LN_IC50']

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

Input = [
        ('polynomial', PolynomialFeatures(include_bias=False)),
        ('scaler', StandardScaler()),
        ('model', Ridge())
        ]
param_grid = {'polynomial__degree':[2, 3],
              'model__alpha': [0.01, 0.1, 1, 10, 100]
              }
pipe = Pipeline(Input)
gridsearch = GridSearchCV(pipe, param_grid, cv=3, scoring='r2', n_jobs=-1, verbose=1)

start_time = time.time()
gridsearch.fit(X_train, y_train)
end_time = time.time()
time_before = end_time - start_time
print(f"Training time before feature selection with Ridge Regression: {time_before:.4f} seconds")

best_model = gridsearch.best_estimator_
print("Best parameters:", gridsearch.best_params_)
print("Best train R^2 score:", gridsearch.best_score_)

test_preds = best_model.predict(X_test)
r2_test = r2_score(y_test, test_preds)
rmse_test = np.sqrt(mean_squared_error(y_test, test_preds))

print(f"Final Test R^2 score: {r2_test:.4f}")
print(f"Final Test RMSE: {rmse_test:.4f}")

Small difference from Linear regression

<h4><b>6.4.3</b> Lasso Regression</h4>

In [ ]:
X_train = train_encoded.drop('LN_IC50', axis=1)
y_train = train_encoded['LN_IC50']
X_test = test_encoded.drop('LN_IC50', axis=1)
y_test = test_encoded['LN_IC50']

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

Input = [
        ('polynomial', PolynomialFeatures(include_bias=False)),
        ('scaler', StandardScaler()),
        ('model', Lasso())
        ]
pipe = Pipeline(Input)
param_grid = {'polynomial__degree':[2, 3],
              'model__alpha': [0.01, 0.1, 1, 10]
              }
gridsearch = GridSearchCV(pipe, param_grid, cv=3, scoring='r2', n_jobs=-1, verbose=1)
start_time = time.time()
gridsearch.fit(X_train, y_train)
end_time = time.time()
time_before = end_time - start_time
print(f"Training time before feature selection with Lasso Regression: {time_before:.4f} seconds")
best_model = gridsearch.best_estimator_
print("Best parameters:", gridsearch.best_params_)
print("Best train R^2 score:", gridsearch.best_score_)
test_preds = best_model.predict(X_test)
r2_test = r2_score(y_test, test_preds)
rmse_test = np.sqrt(mean_squared_error(y_test, test_preds))
print(f"Final Test R^2 score: {r2_test:.4f}")
print(f"Final Test RMSE: {rmse_test:.4f}")

In essence all the Linear models, with regulirization or not, perform less than non-linear models.

Let's try SVR now, which is great for high-dimensional, non-linear data and is less sensitive to outliers.

<h3><b>6.5</b> SVR</h3>

In [ ]:
X_train = train_encoded.drop('LN_IC50', axis=1)
y_train = train_encoded['LN_IC50']
X_test = test_encoded.drop('LN_IC50', axis=1)
y_test = test_encoded['LN_IC50']

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

Input = [
        ('polynomial', PolynomialFeatures(include_bias=False)),
        ('scaler', StandardScaler()),
        ('model', SVR())
        ]
pipe = Pipeline(Input)
param_grid = {'polynomial__degree':[2],
              'model__C': [ 0.1, 1],
              'model__kernel': ["rbf"],
              'model__epsilon': [0.1, 1]
              }
gridsearch = GridSearchCV(pipe, param_grid, cv=3, scoring='r2', n_jobs=-1, verbose=1)
start_time = time.time()
gridsearch.fit(X_train, y_train)
end_time = time.time()
time_before = end_time - start_time
print(f"Training time before feature selection with SVR: {time_before:.4f} seconds")
best_model = gridsearch.best_estimator_
print("Best parameters:", gridsearch.best_params_)
print("Best train R^2 score:", gridsearch.best_score_)
test_preds = best_model.predict(X_test)
r2_test = r2_score(y_test, test_preds)
rmse_test = np.sqrt(mean_squared_error(y_test, test_preds))
print(f"Final Test R^2 score: {r2_test:.4f}")
print(f"Final Test RMSE: {rmse_test:.4f}")

Since Random Forest is the best performing up to now, but relatively slow, I'll try XGBoost which is faster and more accurate, especially for large data sets.

<h3><b>6.6</b> XGBoost</h3>

In [ ]:
X_train = train_encoded.drop('LN_IC50', axis=1)
y_train = train_encoded['LN_IC50']
X_test = test_encoded.drop('LN_IC50', axis=1)
y_test = test_encoded['LN_IC50']

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

Input = [
        ('polynomial', PolynomialFeatures(include_bias=False)),
        ('scaler', StandardScaler()),
        ('model', XGBRegressor(random_state=42))
]
pipe = Pipeline(Input)
param_grid = {
    'model__n_estimators': [1000],
    'model__max_depth': [ None],
    'model__learning_rate': [ 0.1],
    'model__subsample': [ 1.0],
    'model__colsample_bytree': [1.0]
  }

gridsearch = GridSearchCV(pipe, param_grid, cv=3, scoring='r2', n_jobs=-1, verbose=1)
start_time = time.time()
gridsearch.fit(X_train, y_train)
end_time = time.time()
time_before = end_time - start_time
print(f"Training time before feature selection with XGBoost: {time_before:.4f} seconds")
best_model = gridsearch.best_estimator_
print("Best parameters:", gridsearch.best_params_)
print("Best train R^2 score:", gridsearch.best_score_)
test_preds = best_model.predict(X_test)
r2_test = r2_score(y_test, test_preds)
rmse_test = np.sqrt(mean_squared_error(y_test, test_preds))
print(f"Final Test R^2 score: {r2_test:.4f}")
print(f"Final Test RMSE: {rmse_test:.4f}")

The best combination of accuracy and time!

Since XGBoost is the best model since now I'll try the corresponding model from SnapML which will reasonably will give the same accuracy with faster training time.

<h3><b>6.7</b> SBMRegression</h3>

In [ ]:
X_train = train_encoded.drop('LN_IC50', axis=1)
y_train = train_encoded['LN_IC50']
X_test = test_encoded.drop('LN_IC50', axis=1)
y_test = test_encoded['LN_IC50']

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

Input = [
        ('polynomial', PolynomialFeatures(degree=2, include_bias=False)),
        ('scaler', StandardScaler()),
        ('model', SnapBoostingMachineRegressor(random_state=42))
]
pipe = Pipeline(Input)
param_grid = {
     'model__max_depth': [10],
    'model__learning_rate': [0.1],
    'model__subsample': [ 0.8],
    'model__colsample_bytree': [ 0.8],
}
gridsearch = GridSearchCV(pipe, param_grid, cv=3, scoring='r2', n_jobs=-1, verbose=1)
start_time = time.time()
gridsearch.fit(X_train, y_train)
end_time = time.time()
time_before = end_time - start_time
print(f"Training time before feature selection with SnapML: {time_before:.4f} seconds")
best_model = gridsearch.best_estimator_
print("Best parameters:", gridsearch.best_params_)
print("Best train R^2 score:", gridsearch.best_score_)
test_preds = best_model.predict(X_test)
r2_test = r2_score(y_test, test_preds)
rmse_test = np.sqrt(mean_squared_error(y_test, test_preds))
print(f"Final Test R^2 score: {r2_test:.4f}")
print(f"Final Test RMSE: {rmse_test:.4f}")

LightGBM is another reasonable choice since XGBoost performed very well, but we want faster times.

<h3><b>6.8</b> LightGBM</h3>

In [ ]:
X_train = train_encoded.drop('LN_IC50', axis=1)
y_train = train_encoded['LN_IC50']
X_test = test_encoded.drop('LN_IC50', axis=1)
y_test = test_encoded['LN_IC50']

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

Input = [
        ('polynomial', PolynomialFeatures(include_bias=False)),
        ('scaler', StandardScaler()),
        ('model', LGBMRegressor(random_state=42))
]
pipe = Pipeline(Input)
param_grid = {
    'model__n_estimators': [1000],
    'model__max_depth': [None],
    'model__learning_rate': [0.1, 0.4],
    'model__subsample': [0.2]
}
gridsearch = GridSearchCV(pipe, param_grid, cv=3, scoring="r2", n_jobs=-1, verbose=1)
start_time = time.time()
gridsearch.fit(X_train, y_train)
end_time = time.time()
time_before = end_time - start_time
print(f"Training time before feature selection with LightGBM: {time_before:.4f} seconds")
best_model = gridsearch.best_estimator_
print("Best parameters:", gridsearch.best_params_)
print("Best train R^2 score:", gridsearch.best_score_)
test_preds = best_model.predict(X_test)
r2_test = r2_score(y_test, test_preds)
rmse_test = np.sqrt(mean_squared_error(y_test, test_preds))
print(f"Final Test R^2 score: {r2_test:.4f}")
print(f"Final Test RMSE: {rmse_test:.4f}")

Let's try KNN regression, although it's slow for large datasets.

<h3><b>6.9</b> KNN Regression</h3>

In [ ]:
X_train = train_encoded.drop('LN_IC50', axis=1)
y_train = train_encoded['LN_IC50']
X_test = test_encoded.drop('LN_IC50', axis=1)
y_test = test_encoded['LN_IC50']

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

Input = [
        ('polynomial', PolynomialFeatures(degree=2, include_bias=False)),
        ('scaler', StandardScaler()),
        ('model', KNeighborsRegressor())
        ]
pipe = Pipeline(Input)
param_grid = {
    'model__n_neighbors': [3, 5, 7, 10],
    'model__weights': ['uniform', 'distance'],
    'model__p': [1, 2],
    'model__metric': ['euclidean', 'manhattan']
}
gridsearch = GridSearchCV(pipe, param_grid, cv=3, scoring='r2', n_jobs=-1, verbose=1)
start_time = time.time()
gridsearch.fit(X_train, y_train)
end_time = time.time()
time_before = end_time - start_time
print(f"Training time before feature selection with KNN: {time_before:.4f} seconds")
best_model = gridsearch.best_estimator_
print("Best parameters:", gridsearch.best_params_)
print("Best train R^2 score:", gridsearch.best_score_)
test_preds = best_model.predict(X_test)
r2_test = r2_score(y_test, test_preds)
rmse_test = np.sqrt(mean_squared_error(y_test, test_preds))
print(f"Final Test R^2 score: {r2_test:.4f}")
print(f"Final Test RMSE: {rmse_test:.4f}")


Since we have non-linear relationships let's also try Neural Networks

<h3><b>6.10</b> MLPRegressor</h3>

In [ ]:
X_train = train_encoded.drop('LN_IC50', axis=1)
y_train = train_encoded['LN_IC50']
X_test = test_encoded.drop('LN_IC50', axis=1)
y_test = test_encoded['LN_IC50']

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

Input = [
        ('polynomial', PolynomialFeatures(include_bias=False)),
        ('scaler', StandardScaler()),
        ('model', MLPRegressor(random_state=42))
        ]
pipe = Pipeline(Input)
parma_grid = {
    'model__hidden_layer_sizes': [(50,), (100,), (50, 50)],
    'model__activation': ['relu', 'tanh'],
    'model__solver': ['adam', 'sgd'],
    'model__alpha': [0.0001, 0.001, 0.01],
    'model__learning_rate': ['constant', 'adaptive']
}
gridsearch = GridSearchCV(pipe, parma_grid, cv=3, scoring='r2', n_jobs=-1, verbose=1)
start_time = time.time()
gridsearch.fit(X_train, y_train)
end_time = time.time()
time_before = end_time - start_time
print(f"Training time before feature selection with MLPRegressor: {time_before:.4f} seconds")
best_model = gridsearch.best_estimator_
print("Best parameters:", gridsearch.best_params_)
print("Best train R^2 score:", gridsearch.best_score_)
test_preds = best_model.predict(X_test)
r2_test = r2_score(y_test, test_preds)
rmse_test = np.sqrt(mean_squared_error(y_test, test_preds))
print(f"Final Test R^2 score: {r2_test:.4f}")
print(f"Final Test RMSE: {rmse_test:.4f}")

<h2><b>7.</b> Conclusion</h2>



After data importing, preprocessing, encoding, EDA etc the best performing models were XGBoost and LightGBM, with a test accuracy of 0.999. Obviously, with more memory you can do a wider grid search, more cv folds etc and achieve an even better score. Decision trees was the fastest, but with less accuracy. Given the magnitude and non-linearity of the data set, the choice of random forest-like models was apparent. Following Samira Alipour while checking for outliers and skewness I grouped the dataset on drugs. However, in my approach, I also grouped for drugs per specific tissue and condition, which narrowed down the the range of outliers and skewness. In fact, the skewness was that low which dictated that I shouldn't move forward in handling it, since I would loose valuable info (I include the code for handling outliers and skewnees, even though I don't use it).




I hope you find this interesting and helpfull.

Thank you!


Ektoras Delaportas  
Chemist MSc, Pharmacist MPharm